# Use the letters for classification 
Later should use the segmented created pages to pretrain the model
And the actual dataset to fine tune it

In [21]:
from data_management.loadHabbakuk import habbakukLettersDataset
from data_management.datasetSplitter import datasetSplitter
from segment import Segment

AugmentedFolder = "Data/Habbakuk_Letters"

# options on whether or not you want the labels and/or the BBs (both false also works)
Aug_dataset = habbakukLettersDataset(folder_path=AugmentedFolder)
BATCH_SIZE = 16
train_loader, validation_loader = datasetSplitter(Aug_dataset, batch_size=BATCH_SIZE, validation_split=0.2)


In [22]:
# Monkbril2 train, test, validation sets

"""

from data_management.loadDSSCharacters import dssLettersDataset
from data_management.datasetSplitter import datasetSplitter, collate_fn

from torch.utils.data import DataLoader

BATCH_SIZE = 16
shuffle_loaders = True


dataSource = "monkbrill2" # name of the folder, works with "monkbrill" as well

train_folder = "Data/dssLetters/train/"
val_folder = "Data/dssLetters/validation/"
test_folder = "Data/dssLetters/test/"

train_dataset = dssLettersDataset(folder_path=train_folder)
val_dataset = dssLettersDataset(folder_path=val_folder)
test_dataset = dssLettersDataset(folder_path=train_folder)


train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=shuffle_loaders)
validation_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=shuffle_loaders)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=shuffle_loaders)


characterClasses = train_dataset.characterClasses

"""

'\n\nfrom data_management.loadDSSCharacters import dssLettersDataset\nfrom data_management.datasetSplitter import datasetSplitter, collate_fn\n\nfrom torch.utils.data import DataLoader\n\nBATCH_SIZE = 16\nshuffle_loaders = True\n\n\ndataSource = "monkbrill2" # name of the folder, works with "monkbrill" as well\n\ntrain_folder = "Data/dssLetters/train/"\nval_folder = "Data/dssLetters/validation/"\ntest_folder = "Data/dssLetters/test/"\n\ntrain_dataset = dssLettersDataset(folder_path=train_folder)\nval_dataset = dssLettersDataset(folder_path=val_folder)\ntest_dataset = dssLettersDataset(folder_path=train_folder)\n\n\ntrain_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=shuffle_loaders)\nvalidation_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=shuffle_loaders)\ntest_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=shuffle_loaders)\n\n\ncharacterClasses = train_dataset.

In [23]:
input, target = next(iter(train_loader))
input[0].shape

torch.Size([1, 50, 50])

Later try to do this

In [24]:
#import os
#segment = Segment()
#segment.segment_characters(real_data=True, generated_data=False)


The arquitecture has been decided very arbitrarily for now

In [25]:
import torch
import torch.nn as nn

class CharacterCNN(nn.Module):
    def __init__(self, numChannels = 1, classes = 27, BATCH_SIZE = 16, dropout_rate = 0.5):
        super(CharacterCNN, self).__init__()

        """
        self.conv1 = nn.Conv2d(numChannels, 32, kernel_size=5, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, BATCH_SIZE, kernel_size=5, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear( BATCH_SIZE *12* 12, 128) #this shoulf be BATCH_SIZE *12* 12 but that doesnt work
        self.fc2 = nn.Linear(128, classes)
        self.relu = nn.ReLU() # could also just use nn.functional.relu
        #self.logSoftmax = nn.LogSoftmax(dim=1)
        """

        self.conv1 = nn.Conv2d(numChannels, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, BATCH_SIZE, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear( BATCH_SIZE *12* 12, 64) #this shoulf be BATCH_SIZE *12* 12 but that doesnt work
        self.fc2 = nn.Linear(64, classes)
        self.relu = nn.ReLU() # could also just use nn.functional.relu
        #self.logSoftmax = nn.LogSoftmax(dim=1)
        self.dropout = nn.Dropout(dropout_rate)


 
    # Using nn.functional provides a more concise syntax since it directly applies the operations as functions,
    # while using the corresponding layers from torch.nn allows for explicit control over the layers used in the network
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.dropout(x)
        x = self.pool(self.relu(self.conv2(x)))
        x = self.dropout(x)
        #x = torch.flatten(x, 1)
        x = x.view(x.size(0), -1) #same as flatten
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        output = self.fc2(x)
        #output = self.logSoftmax(output)
        return output

# Create an instance of the CNN model
model = CharacterCNN()

# Print the model architecture



class DanNet1(nn.Module):
    def __init__(self, num_classes=27, batch_size = 16, dropout_rate = 0.5):
        super(DanNet1, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, batch_size, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm2d(batch_size),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Linear(batch_size *24 *24, 128)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(128, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        print(out.shape)
        out = out.reshape(out.size(0), -1)
        print(out.shape)
        out = self.fc(out)
        out = self.relu(out)
        out = self.fc1(out)
        return out


Training

In [26]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg") # ??¿¿

# import the necessary packages
from sklearn.metrics import classification_report
from torchvision.transforms import ToTensor
from torch.optim import Adam
import matplotlib.pyplot as plt
import numpy as np
import argparse
import torch
import time
import os


In [27]:
"""
# construct the argument parser and parse the arguments
model_dir = "Model1"
os.makedirs(model_dir, exist_ok = True)

ap = argparse.ArgumentParser()
ap.add_argument("-m", "--model", type=str, required=True,
	help=model_dir)
ap.add_argument("-p", "--plot", type=str, required=True,
	help=model_dir)
args = vars(ap.parse_args())
"""

'\n# construct the argument parser and parse the arguments\nmodel_dir = "Model1"\nos.makedirs(model_dir, exist_ok = True)\n\nap = argparse.ArgumentParser()\nap.add_argument("-m", "--model", type=str, required=True,\n\thelp=model_dir)\nap.add_argument("-p", "--plot", type=str, required=True,\n\thelp=model_dir)\nargs = vars(ap.parse_args())\n'

In [28]:
# define training hyperparameters
INIT_LR = 1e-3
BATCH_SIZE = 16
EPOCHS = 100

# calculate steps per epoch for training and validation set
trainSteps = len(train_loader.dataset) // BATCH_SIZE
valSteps = len(validation_loader.dataset) // BATCH_SIZE


# set the device we will be using to train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [29]:
# initialize the  model

model = DanNet1().to(device)
# initialize our optimizer and loss function
opt = Adam(model.parameters(), lr=INIT_LR)
criterion = nn.CrossEntropyLoss()
#criterion = nn.NLLLoss() #When we combine the nn.NLLoss class with LogSoftmax in our model definition, we arrive at categorical cross-entropy loss
#  (which is the equivalent to training a model with an output Linear layer and an nn.CrossEntropyLoss loss).
# initialize a dictionary to store training history
H = {
	"train_loss": [],
	"train_acc": [],
	"val_loss": [],
	"val_acc": []
}
# measure how long training is going to take
startTime = time.time()

Training loop

In [30]:
# loop over our epochs

for e in range(0, EPOCHS):
	# set the model in training mode
	model.train()
	# initialize the total training and validation loss
	totalTrainLoss = 0
	totalValLoss = 0
	# initialize the number of correct predictions in the training
	# and validation step
	trainCorrect = 0
	valCorrect = 0
	# loop over the training set
	for (x, y) in train_loader:
		x = torch.stack([image.to(device) for image in x])
		y = torch.stack([torch.LongTensor([target]) for target in y])
		y = torch.squeeze(y)

		output = model(x)
		loss = criterion(output, y)

		# perform a forward pass and calculate the training loss
		
		# zero out the gradients, perform the backpropagation step,
		# and update the weights
		opt.zero_grad()
		loss.backward()
		opt.step()
		# add the loss to the total training loss so far and
		# calculate the number of correct predictions
		totalTrainLoss += loss
		trainCorrect += (output.argmax(1) == y).type(
			torch.float).sum().item()

		
	if e % 10 == 0:

		"EVALUATION"

		# switch off autograd for evaluation
		with torch.no_grad():
			# set the model in evaluation mode
			model.eval()
			
			# loop over the validation set
			for (x, y) in validation_loader:
				x = torch.stack([image.to(device) for image in x])
				y = torch.stack([torch.LongTensor([target]) for target in y])
				y = torch.squeeze(y)
				output = model(x)
				
				totalValLoss += criterion(output, y)
				
				# calculate the number of correct predictions
				valCorrect += (output.argmax(1) == y).type(torch.float).sum().item()

		# calculate the average training and validation loss
		avgTrainLoss = totalTrainLoss / trainSteps
		avgValLoss = totalValLoss / valSteps
		# calculate the training and validation accuracy
		trainCorrect = trainCorrect / len(train_loader.dataset)
		valCorrect = valCorrect / len(validation_loader.dataset)
		# update our training history
		H["train_loss"].append(avgTrainLoss.cpu().detach().numpy())
		H["train_acc"].append(trainCorrect)
		H["val_loss"].append(avgValLoss.cpu().detach().numpy())
		H["val_acc"].append(valCorrect)

		# print the model training and validation information
		print("[INFO] EPOCH: {}/{}".format(e + 1, EPOCHS))
		print("Train loss: {:.6f}, Train accuracy: {:.4f}".format(
			avgTrainLoss, trainCorrect))
		print("Val loss: {:.6f}, Val accuracy: {:.4f}\n".format(
			avgValLoss, valCorrect))



torch.Size([16, 16, 24, 24])
torch.Size([16, 9216])
torch.Size([16, 16, 24, 24])
torch.Size([16, 9216])
torch.Size([16, 16, 24, 24])
torch.Size([16, 9216])
torch.Size([16, 16, 24, 24])
torch.Size([16, 9216])
torch.Size([16, 16, 24, 24])
torch.Size([16, 9216])
torch.Size([16, 16, 24, 24])
torch.Size([16, 9216])
torch.Size([16, 16, 24, 24])
torch.Size([16, 9216])
torch.Size([16, 16, 24, 24])
torch.Size([16, 9216])
torch.Size([16, 16, 24, 24])
torch.Size([16, 9216])
torch.Size([16, 16, 24, 24])
torch.Size([16, 9216])
torch.Size([16, 16, 24, 24])
torch.Size([16, 9216])
torch.Size([16, 16, 24, 24])
torch.Size([16, 9216])
torch.Size([16, 16, 24, 24])
torch.Size([16, 9216])
torch.Size([8, 16, 24, 24])
torch.Size([8, 9216])
torch.Size([16, 16, 24, 24])
torch.Size([16, 9216])
torch.Size([16, 16, 24, 24])
torch.Size([16, 9216])
torch.Size([16, 16, 24, 24])
torch.Size([16, 9216])
torch.Size([6, 16, 24, 24])
torch.Size([6, 9216])
[INFO] EPOCH: 1/100
Train loss: 3.273650, Train accuracy: 0.0741
Val

KeyboardInterrupt: 

Statistics

In [ ]:
"""
# calculate the average training and validation loss
avgTrainLoss = totalTrainLoss / trainSteps
avgValLoss = totalValLoss / valSteps
# calculate the training and validation accuracy
trainCorrect = trainCorrect / len(train_loader.dataset)
valCorrect = valCorrect / len(validation_loader.dataset)
# update our training history
H["train_loss"].append(avgTrainLoss.cpu().detach().numpy())
H["train_acc"].append(trainCorrect)
H["val_loss"].append(avgValLoss.cpu().detach().numpy())
H["val_acc"].append(valCorrect)
"""
# print the model training and validation information
print("[INFO] EPOCH: {}/{}".format(e + 1, EPOCHS))
print("Train loss: {:.6f}, Train accuracy: {:.4f}".format(
    avgTrainLoss, trainCorrect))
print("Val loss: {:.6f}, Val accuracy: {:.4f}\n".format(
    avgValLoss, valCorrect))


[INFO] EPOCH: 100/100
Train loss: 0.568179, Train accuracy: 170.0000
Val loss: 0.834987, Val accuracy: 0.0000



PLOTS

In [ ]:
# only saving one for now
os.makedirs("classification_models", exist_ok = True)

args = {
    "plot_loss": "classification_models/plot_loss.png",
    "plot_acc": "classification_models/plot_acc.png",
    "model": "classification_models/model.pth"
}

In [ ]:
# plot the training loss and accuracy
import numpy as np

epochs = np.arange(len(H["train_loss"]))*10

plt.style.use("ggplot")
plt.figure()
plt.plot(epochs, H["train_loss"],  label="train_loss")
plt.plot(epochs, H["val_loss"], label="val_loss")
plt.title("Training and Validation Loss on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend()
plt.savefig(args["plot_loss"])

plt.style.use("ggplot")
plt.figure()
plt.plot(epochs, H["train_acc"], label="train_acc")
plt.plot(epochs, H["val_acc"], label="val_acc")
plt.title("Training and Validation Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Accuracy")
plt.legend()
plt.savefig(args["plot_acc"])
# serialize the model to disk
torch.save(model, args["model"])